In [1]:
#import matplotlib.pyplot as plt
#matplotlib.use("Agg")


from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
class MiniVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # first CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))

        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # second CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))

        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model
    
#%%


In [4]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
print(trainX.shape)
print(testX.shape)

[INFO] loading CIFAR-10 data...
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [6]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer","dog", "frog", "horse", "ship", "truck"]

In [8]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay=0.01 / 10, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
no_epochs=10
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=64, epochs=no_epochs, verbose=1)

[INFO] compiling model...
[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 190s - loss: 1.8229 - acc: 0.3328 - val_loss: 1.4515 - val_acc: 0.4677
Epoch 2/10
50000/50000 [==============================] - 182s - loss: 1.3981 - acc: 0.4930 - val_loss: 1.2184 - val_acc: 0.5643
Epoch 3/10
50000/50000 [==============================] - 185s - loss: 1.2206 - acc: 0.5651 - val_loss: 1.0846 - val_acc: 0.6133
Epoch 4/10
50000/50000 [==============================] - 185s - loss: 1.1270 - acc: 0.5978 - val_loss: 1.0279 - val_acc: 0.6369
Epoch 5/10
50000/50000 [==============================] - 186s - loss: 1.0579 - acc: 0.6233 - val_loss: 0.9740 - val_acc: 0.6560
Epoch 6/10
50000/50000 [==============================] - 184s - loss: 1.0032 - acc: 0.6420 - val_loss: 0.9422 - val_acc: 0.6647
Epoch 7/10
50000/50000 [==============================] - 181s - loss: 0.9650 - acc: 0.6596 - val_loss: 0.8993 - val_acc: 0

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...


In [ ]:
# plot the training loss and accuracy
plt.style.use("seaborn-white")
plt.figure()
plt.plot(np.arange(0, no_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, no_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, no_epochs), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, no_epochs), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend()